# 소리함 프로젝트 :  음성합성 데모

이 문서는 인공지능및응용 수업의 소리함 프로젝트의 음성 합성 문서입니다.

## 1. 구글 드라이브 마운트

음성합성을 위해 학습한 모델이 있는 구글 드라이브를 마운트합니다.  
마운트할 구글 드라이브 내에 다음 파일들이 존재해야합니다.

- `/Colab Notebooks/data/glowtts-v2/model_file.pth.tar`
- `/Colab Notebooks/data/glowtts-v2/config.json`
- `/Colab Notebooks/data/hifigan-v2/model_file.pth.tar`
- `/Colab Notebooks/data/hifigan-v2/config.json`

만약 아래에 `Enter your authorization code:`과 같은 메시지가 출력될 경우,  
같이 출력된 링크에 접속하여, 마운트할 구글 계정을 선택하신 후, 인증 코드를 복사하여 인증해야 합니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. 필수 라이브러리 및 함수 불러오기

실행에 필요한 라이브러리 및 함수를 불러옵니다.

In [ ]:
import os
import sys
from pathlib import Path

In [ ]:
%cd /content
!git clone --depth 1 https://github.com/sce-tts/TTS.git -b sce-tts
!git clone --depth 1 https://github.com/sce-tts/g2pK.git
%cd /content/TTS
!pip install folium==0.2.1
!pip install imgaug==0.2.5
!pip install -q --no-cache-dir -e .
%cd /content/g2pK
!pip install -q --no-cache-dir "konlpy" "jamo" "nltk" "python-mecab-ko"
!pip install -q --no-cache-dir -e .

In [ ]:
%cd /content/g2pK
import g2pk
g2p = g2pk.G2p()

In [ ]:
%cd /content/TTS
import re
import sys
from unicodedata import normalize
import IPython
from TTS.utils.synthesizer import Synthesizer


def normalize_text(text):
    text = text.strip()

    for c in ",;:":
        text = text.replace(c, ".")
    text = remove_duplicated_punctuations(text)

    text = jamo_text(text)

    text = g2p.idioms(text)
    text = g2pk.english.convert_eng(text, g2p.cmu)
    text = g2pk.utils.annotate(text, g2p.mecab)
    text = g2pk.numerals.convert_num(text)
    text = re.sub("/[PJEB]", "", text)

    text = alphabet_text(text)

    # remove unreadable characters
    text = normalize("NFD", text)
    text = "".join(c for c in text if c in symbols)
    text = normalize("NFC", text)

    text = text.strip()
    if len(text) == 0:
        return ""

    # only single punctuation
    if text in '.!?':
        return punctuation_text(text)

    # append punctuation if there is no punctuation at the end of the text
    if text[-1] not in '.!?':
        text += '.'

    return text


def remove_duplicated_punctuations(text):
    text = re.sub(r"[.?!]+\?", "?", text)
    text = re.sub(r"[.?!]+!", "!", text)
    text = re.sub(r"[.?!]+\.", ".", text)
    return text


def split_text(text):
    text = remove_duplicated_punctuations(text)

    texts = []
    for subtext in re.findall(r'[^.!?\n]*[.!?\n]', text):
        texts.append(subtext.strip())

    return texts


def alphabet_text(text):
    text = re.sub(r"(a|A)", "에이", text)
    text = re.sub(r"(b|B)", "비", text)
    text = re.sub(r"(c|C)", "씨", text)
    text = re.sub(r"(d|D)", "디", text)
    text = re.sub(r"(e|E)", "이", text)
    text = re.sub(r"(f|F)", "에프", text)
    text = re.sub(r"(g|G)", "쥐", text)
    text = re.sub(r"(h|H)", "에이치", text)
    text = re.sub(r"(i|I)", "아이", text)
    text = re.sub(r"(j|J)", "제이", text)
    text = re.sub(r"(k|K)", "케이", text)
    text = re.sub(r"(l|L)", "엘", text)
    text = re.sub(r"(m|M)", "엠", text)
    text = re.sub(r"(n|N)", "엔", text)
    text = re.sub(r"(o|O)", "오", text)
    text = re.sub(r"(p|P)", "피", text)
    text = re.sub(r"(q|Q)", "큐", text)
    text = re.sub(r"(r|R)", "알", text)
    text = re.sub(r"(s|S)", "에스", text)
    text = re.sub(r"(t|T)", "티", text)
    text = re.sub(r"(u|U)", "유", text)
    text = re.sub(r"(v|V)", "브이", text)
    text = re.sub(r"(w|W)", "더블유", text)
    text = re.sub(r"(x|X)", "엑스", text)
    text = re.sub(r"(y|Y)", "와이", text)
    text = re.sub(r"(z|Z)", "지", text)

    return text


def punctuation_text(text):
    # 문장부호
    text = re.sub(r"!", "느낌표", text)
    text = re.sub(r"\?", "물음표", text)
    text = re.sub(r"\.", "마침표", text)

    return text


def jamo_text(text):
    # 기본 자모음
    text = re.sub(r"ㄱ", "기역", text)
    text = re.sub(r"ㄴ", "니은", text)
    text = re.sub(r"ㄷ", "디귿", text)
    text = re.sub(r"ㄹ", "리을", text)
    text = re.sub(r"ㅁ", "미음", text)
    text = re.sub(r"ㅂ", "비읍", text)
    text = re.sub(r"ㅅ", "시옷", text)
    text = re.sub(r"ㅇ", "이응", text)
    text = re.sub(r"ㅈ", "지읒", text)
    text = re.sub(r"ㅊ", "치읓", text)
    text = re.sub(r"ㅋ", "키읔", text)
    text = re.sub(r"ㅌ", "티읕", text)
    text = re.sub(r"ㅍ", "피읖", text)
    text = re.sub(r"ㅎ", "히읗", text)
    text = re.sub(r"ㄲ", "쌍기역", text)
    text = re.sub(r"ㄸ", "쌍디귿", text)
    text = re.sub(r"ㅃ", "쌍비읍", text)
    text = re.sub(r"ㅆ", "쌍시옷", text)
    text = re.sub(r"ㅉ", "쌍지읒", text)
    text = re.sub(r"ㄳ", "기역시옷", text)
    text = re.sub(r"ㄵ", "니은지읒", text)
    text = re.sub(r"ㄶ", "니은히읗", text)
    text = re.sub(r"ㄺ", "리을기역", text)
    text = re.sub(r"ㄻ", "리을미음", text)
    text = re.sub(r"ㄼ", "리을비읍", text)
    text = re.sub(r"ㄽ", "리을시옷", text)
    text = re.sub(r"ㄾ", "리을티읕", text)
    text = re.sub(r"ㄿ", "리을피읍", text)
    text = re.sub(r"ㅀ", "리을히읗", text)
    text = re.sub(r"ㅄ", "비읍시옷", text)
    text = re.sub(r"ㅏ", "아", text)
    text = re.sub(r"ㅑ", "야", text)
    text = re.sub(r"ㅓ", "어", text)
    text = re.sub(r"ㅕ", "여", text)
    text = re.sub(r"ㅗ", "오", text)
    text = re.sub(r"ㅛ", "요", text)
    text = re.sub(r"ㅜ", "우", text)
    text = re.sub(r"ㅠ", "유", text)
    text = re.sub(r"ㅡ", "으", text)
    text = re.sub(r"ㅣ", "이", text)
    text = re.sub(r"ㅐ", "애", text)
    text = re.sub(r"ㅒ", "얘", text)
    text = re.sub(r"ㅔ", "에", text)
    text = re.sub(r"ㅖ", "예", text)
    text = re.sub(r"ㅘ", "와", text)
    text = re.sub(r"ㅙ", "왜", text)
    text = re.sub(r"ㅚ", "외", text)
    text = re.sub(r"ㅝ", "워", text)
    text = re.sub(r"ㅞ", "웨", text)
    text = re.sub(r"ㅟ", "위", text)
    text = re.sub(r"ㅢ", "의", text)

    return text


def normalize_multiline_text(long_text):
    texts = split_text(long_text)
    normalized_texts = [normalize_text(text).strip() for text in texts]
    return [text for text in normalized_texts if len(text) > 0]

def synthesize(text):
    wavs = synthesizer.tts(text, None, None)
    return wavs

## 3. 학습한 모델 불러오기

학습한 Glow-TTS와 HiFi-GAN 모델을 불러옵니다.

만약 다른 체크포인트에서 불러오시려면 아래 코드에서 경로를 아래와 같이 적절하게 수정합니다.

가장 최신의 학습된 버전을 불러와야 한다.

Example)
- Glow-TTS : best_model.pth.tar
- HiFigan-TTS :  best_model_000000.pth.tar

```python
synthesizer = Synthesizer(
    "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/glowtts-v2-December-1-2021_08+17AM-d897f2e/best_model.pth.tar",
    "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/glowtts-v2-December-1-2021_08+17AM-d897f2e/config.json",
    None,
    "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/hifigan-v2-December-1-2021_08+26AM-d897f2e/best_model.pth.tar_000000.pth.tar",
    "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/hifigan-v2-December-1-2021_08+26AM-d897f2e/config.json",
    None,
    None,
    False,
)
```

In [ ]:
synthesizer = Synthesizer(
    "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/glowtts-v2-December-01-2021_08+42AM-3aa165a/best_model.pth.tar",
    "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/glowtts-v2-December-01-2021_08+42AM-3aa165a/config.json",
    None,
    "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/hifigan-v2-December-01-2021_02+00PM-3aa165a/best_model_298813.pth.tar",
    "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/hifigan-v2-December-01-2021_02+00PM-3aa165a/config.json",
    None,
    None,
    False,
)
symbols = synthesizer.tts_config.characters.characters

## 4. 음성 합성

실제 음성 합성을 수행합니다.

`texts` 내부의 값을 변경하여 다른 문장의 합성도 시도해보실 수 있습니다.

마침표(.)는 문장의 끝을 의미하므로 두 문장을 연달아 적을시에도 마침표(.)가 있으면 2개의 문장으로 인식합니다.

그러므로 두 개 이상의 문장을 연달아 합성하고 싶을 시에는 마침표를 사용하지 않고 연달아 작성해준다.

In [ ]:

texts = """
한양대학교 정보시스템학과 인공지능및응용 프로젝트 블로그
2021년 겨울은 매우 춥다 그러므로 옷을 따뜻하게 입어야 한다.
안녕하세요 저희는 소리함이라는 개인화 Text to Speech 를 제작하는 작업을 수행하였습니다.
아들 식탁 위에 돈 뒀으니 밥 사먹어.
서울특별시 특허허가과 허가과장 허과장
"""
for text in normalize_multiline_text(texts):
    wav = synthesizer.tts(text, None, None)
    IPython.display.display(IPython.display.Audio(wav, rate=22050))  